### Import libraries

In [26]:
import requests
import pandas as pd

### Fetch data from the API | Census 2022


In [27]:
url = 'https://servicodados.ibge.gov.br/api/v3/agregados/4712/periodos/2022/variaveis/382?localidades=N6[all]'
response = requests.get(url)

### Ensure the request was successful

In [28]:
if response.status_code == 200:
    data = response.json()
    print("Successful request!")
else:
    print(f"Request failed with status code {response.status_code}")

Successful request!


### Cleanning and transforming the data

In [29]:
data = response.json()

# Flatten data
flattened_data = []
for item in data[0]['resultados'][0]['series']:
    id = item['localidade']['id']
    name = item['localidade']['nome']
    population = item['serie']['2022']
    flattened_data.append({'id': id, 'name': name, 'population': population})

# Create DataFrame
df = pd.DataFrame(flattened_data)
df[['city', 'state']] = df['name'].str.split(' - ', expand=True)

# Convert the population column to integers
df['population'] = df['population'].astype(int)

# Add the 'year' column
df['year'] = 2022

# Drop the 'name' column and reorder columns
df = df[['id', 'year', 'population', 'city', 'state']]

# Load existing data from CSV
csv_path = 'C:\\Users\\jlfen\\OneDrive\\Documentos\\JoaoKasten\\005_applied_projects\\project.censo-dataviz\\data\\census_data.csv'
df_existing = pd.read_csv(csv_path)

# Combine new and existing data
df_combined = pd.concat([df_existing, df], ignore_index=True)

# Remove duplicates based on 'id' and 'year' columns
df_combined.drop_duplicates(subset=['id', 'year'], inplace=True)

# Save the combined DataFrame back to the CSV
df_combined.to_csv(csv_path, index=False, encoding='utf-8-sig')

### Saving _df_ to a csv file